# Broadcasts

This notebook explains the **different types of broadcast available** in PyBaMM.
Understanding of the [expression_tree](./expression-tree.ipynb) and [discretisation](../spatial_methods/finite-volumes.ipynb) notebooks is assumed.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import numpy as np

import pybamm

Note: you may need to restart the kernel to use updated packages.


We also explicitly set up the discretisation that is used for this notebook. We use a small number of points in each domain, in order to easily visualise the results.

In [ ]:
var = pybamm.standard_spatial_vars
geometry = {
    "negative electrode": {var.x_n: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(1)}},
    "negative particle": {var.r_n: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(1)}},
}

submesh_types = {
    "negative electrode": pybamm.Uniform1DSubMesh,
    "negative particle": pybamm.Uniform1DSubMesh,
}

var_pts = {var.x_n: 5, var.r_n: 3}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {
    "negative electrode": pybamm.FiniteVolume(),
    "negative particle": pybamm.FiniteVolume(),
}
disc = pybamm.Discretisation(mesh, spatial_methods)

## Primary broadcasts

In [ ]:
T = pybamm.Variable("T", domain="negative electrode")
disc.set_variable_slices([T])
disc_T = disc.process_symbol(T)
disc_T.evaluate(y=np.linspace(0, 1, 5))

array([[0.  ],
       [0.25],
       [0.5 ],
       [0.75],
       [1.  ]])

We then broadcast `T` onto the "negative particle" domain (using primary broadcast as we are going from the larger electrode scale to the smaller particle scale), and discretise and evaluate the resulting object.

In [ ]:
primary_broad_T = pybamm.PrimaryBroadcast(T, "negative particle")
disc_T = disc.process_symbol(primary_broad_T)
disc_T.evaluate(y=np.linspace(0, 1, 5))

array([[0.  ],
       [0.  ],
       [0.  ],
       [0.25],
       [0.25],
       [0.25],
       [0.5 ],
       [0.5 ],
       [0.5 ],
       [0.75],
       [0.75],
       [0.75],
       [1.  ],
       [1.  ],
       [1.  ]])

The broadcasted object makes 3 (since the r-grid has 3 points) copies of each element of `T` and stacks them all up to give an object with size 3x5=15. In the resulting vector, the first 3 entries correspond to the 3 points in the r-domain at the first x-grid point (where T=0 uniformly in r), the next 3 entries correspond to the next 3 points in the r-domain at the second x-grid point (where T=0.25 uniformly in r), etc

## Secondary broadcasts

Secondary broadcasts are used to broadcast from a "smaller" scale to a "larger" scale, for example broadcasting SPM particle concentrations c_s(r) from the particles to the electrodes. Note that this wouldn't be used to broadcast particle concentrations in the DFN, since these already depend on both x and r.
To demonstrate this, we first create a variable `c_s` on the negative particle domain, discretise it, and evaluate it with a simple linear vector

In [ ]:
c_s = pybamm.Variable("c_s", domain="negative particle")
disc.set_variable_slices([c_s])
disc_c_s = disc.process_symbol(c_s)
disc_c_s.evaluate(y=np.linspace(0, 1, 3))

array([[0. ],
       [0.5],
       [1. ]])

We then broadcast `c_s` onto the "negative electrode" domain (using secondary broadcast as we are going from the smaller particle scale to the large electrode scale), and discretise and evaluate the resulting object.

In [ ]:
secondary_broad_c_s = pybamm.SecondaryBroadcast(c_s, "negative electrode")
disc_broad_c_s = disc.process_symbol(secondary_broad_c_s)
disc_broad_c_s.evaluate(y=np.linspace(0, 1, 3))

array([[0. ],
       [0.5],
       [1. ],
       [0. ],
       [0.5],
       [1. ],
       [0. ],
       [0.5],
       [1. ],
       [0. ],
       [0.5],
       [1. ],
       [0. ],
       [0.5],
       [1. ]])

The broadcasted object makes 5 (since the x-grid has 5 points) identical copies of the whole variable `c_s` to give an object with size 5x3=15. In the resulting vector, the first 3 entries correspond to the 3 points in the r-domain at the first x-grid point (where c_s varies in r), the next 3 entries correspond to the next 3 points in the r-domain at the second x-grid point (where c_s varies in r), etc

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()

[1] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[2] Valentin Sulzer, Scott G. Marquis, Robert Timms, Martin Robinson, and S. Jon Chapman. Python Battery Mathematical Modelling (PyBaMM). Journal of Open Research Software, 9(1):14, 2021. doi:10.5334/jors.309.

